### LLM Chain 만들기


## 1. 환경 구성

### 1) 라이브러리 설치

In [15]:
# poetry add python-dotenv langchain langchain-openai

### 2) OpenAI 인증키 설정
https://openai.com/

In [16]:
from dotenv import load_dotenv
import os
# .env 파일을 불러와서 환경 변수로 설정
load_dotenv(dotenv_path='.env')

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
print(OPENAI_API_KEY[:5])

gsk_E


In [17]:
import langchain
print(langchain.__version__)

0.3.27


## 2. LLM Chain

### 1) Prompt + LLM


In [18]:
from langchain_openai import ChatOpenAI

# model
# llm = ChatOpenAI(model="gpt-3.5-turbo-0125")
llm = ChatOpenAI(
    api_key=OPENAI_API_KEY,
    base_url="https://api.groq.com/openai/v1",  # Groq API 엔드포인트
    model="meta-llama/llama-4-scout-17b-16e-instruct",  # Spring AI와 동일한 모델
    temperature=0.7
)

# chain 실행
result = llm.invoke("인공지능 모델의 학습 원리에 대하여 쉽게 설명해 주세요.")
print(type(result))

BadRequestError: Error code: 400 - {'error': {'message': 'Organization has been restricted. Please reach out to support if you believe this was in error.', 'type': 'invalid_request_error', 'code': 'organization_restricted'}}

In [ ]:
print(result)

content="인공지능 모델의 학습 원리는 사람의 뇌가 학습하는 원리와 유사합니다. 예를 들어, 고양이를 인식하는 모델을 만든다고 가정해 보겠습니다. \n\n1.  **데이터 수집**: 수많은 고양이, 강아지, 새 등의 사진 데이터를 수집합니다. 각 사진에는 '고양이', '강아지', '새'와 같은 레이블(라벨)이 붙어 있습니다.\n2.  **모델 초기화**: 모델은 처음에 무작위로 설정된 가중치를 가지고 시작합니다. 이 가중치는 모델의 신경망에서 각 노드 간의 연결 강도를 결정합니다.\n3.  **예측**: 모델에 새로운 사진이 입력되면, 가중치를 기반으로 '고양이' 또는 '강아지'와 같은 레이블을 예측합니다.\n4.  **오류 계산**: 모델의 예측과 실제 레이블(예: 고양이)을 비교하여 오류를 계산합니다. 이 오류를 손실 함수라고 합니다.\n5.  **가중치 업데이트**: 모델은 손실 함수를 최소화하기 위해 가중치를 조정합니다. 이 과정은 역전파 알고리즘을 통해 이루어집니다. 역전파는 예측 오류가 신경망의 각 가중치에 어떻게 기여하는지 계산하고, 가중치를 조정하여 오류를 줄입니다.\n6.  **반복 학습**: 모델은 많은 사진 데이터에 대해 예측, 오류 계산, 가중치 업데이트를 반복합니다. 이 과정은 모델이 데이터를 잘 학습할 때까지 계속됩니다.\n\n이러한 학습 과정을 통해 모델은 고양이, 강아지, 새 등을 구별하는 법을 배우게 됩니다. 모델이 학습한 후에는 새로운, 보지 못한 사진에 대해서도 높은 확률로 올바른 레이블을 예측할 수 있습니다." additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 360, 'prompt_tokens': 24, 'total_tokens': 384, 'completion_tokens_details': None, 'prompt_tokens_details': None, 'queue_time': 0.200937874, 

In [ ]:
print(result.content)

### 2) PromptTemplate + LLM

In [ ]:
from langchain_core.prompts import PromptTemplate

prompt = PromptTemplate.from_template("You are an expert in AI Expert. Answer the question. <Question>: {input}에 대해 쉽게 설명해주세요.")

print(type(prompt))
print(prompt)

In [ ]:
from langchain_openai import ChatOpenAI

# llm = ChatOpenAI(model="gpt-3.5-turbo-0125")
llm = ChatOpenAI(
    api_key=OPENAI_API_KEY,
    base_url="https://api.groq.com/openai/v1",  # Groq API 엔드포인트
    model="meta-llama/llama-4-scout-17b-16e-instruct",  # Spring AI와 동일한 모델
    temperature=0.7
)
# chain 연결 (LCEL)
chain = prompt | llm
print(type(chain))

# chain 호출
result = chain.invoke({"input": "인공지능 모델의 학습 원리"})
print(type(result))
print(result)

In [ ]:
print(result.content)

### 3) PromptTemplate + LLM(invoke()) + StrOutputParser

In [ ]:
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser

# 1. 컴포넌트 정의
prompt = PromptTemplate.from_template("You are an expert in AI Expert. \
                                      Answer the question. <Question>: {input}에 대해 쉽게 설명해주세요.")

# llm = ChatOpenAI(model="gpt-3.5-turbo")
llm = ChatOpenAI(
    api_key=OPENAI_API_KEY,
    base_url="https://api.groq.com/openai/v1",  # Groq API 엔드포인트
    model="meta-llama/llama-4-scout-17b-16e-instruct",  # Spring AI와 동일한 모델
    temperature=0.7
)

output_parser = StrOutputParser()

# 2. chain 생성 (LCEL)
chain = prompt | llm | output_parser
print(type(chain))

# 3. chain의 invoke 호출
result = chain.invoke({"input": "인공지능 모델의 학습 원리"})
print(type(result))
print(result)

### 4) PromptTemplate + LLM(stream()) + StrOutputParser

In [ ]:

from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser

# 1. 컴포넌트 정의
prompt = PromptTemplate.from_template("You are an expert in AI Expert. \
                                      Answer the question. <Question>: {input}에 대해 쉽게 설명해주세요.")

#llm = ChatOpenAI(model="gpt-3.5-turbo-0125")
lm = ChatOpenAI(
    api_key=OPENAI_API_KEY,
    base_url="https://api.groq.com/openai/v1",  # Groq API 엔드포인트
    model="meta-llama/llama-4-scout-17b-16e-instruct",  # Spring AI와 동일한 모델
    temperature=0.7
)

# chain 연결 (LCEL)
chain = prompt | llm | StrOutputParser()

# 스트리밍 출력을 위한 요청
answer = chain.stream({"input": "인공지능 모델의 학습 원리"})
# 스트리밍 출력
#print(answer)

for token in answer:
    # 스트림에서 받은 데이터의 내용을 출력합니다. 줄바꿈 없이 이어서 출력하고, 버퍼를 즉시 비웁니다.
    print(token, end="", flush=True)

##### 2) Multiple Chains
* Multi Chain을 활용한 영화 추천 및 줄거리 요약 ( 잘 동작하지 않는 코드)

In [ ]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser

# Step 1: 사용자가 입력한 장르에 따라 영화 추천
prompt1 = ChatPromptTemplate.from_template("{genre} 장르에서 추천할 만한 영화를 한 편 알려주세요.")

# Step 2: 추천된 영화의 줄거리를 요약
prompt2 = ChatPromptTemplate.from_template("{movie} 추천한 영화의 제목을 먼저 알려주시고, 줄을 바꾸어서 영화의 정보(제목,감독,캐스팅,줄거리)를 알려 주세요")

# OpenAI 모델 사용
# llm = ChatOpenAI(model="gpt-3.5-turbo-0125")
llm = ChatOpenAI(
    api_key=OPENAI_API_KEY,
    base_url="https://api.groq.com/openai/v1",  # Groq API 엔드포인트
    #model="meta-llama/llama-4-scout-17b-16e-instruct",  # Spring AI와 동일한 모델
    model="moonshotai/kimi-k2-instruct-0905",
    temperature=0.7
)

# 체인 1: 영화 추천 (입력: 장르 → 출력: 영화 제목)
chain1 = prompt1 | llm | StrOutputParser()

# Step 1: 사용자가 입력한 장르에 따라 영화 추천
movie = chain1.invoke({"genre": "Drama"})  # 영화 제목 얻기

print(type(movie))
print(" ===> 추천된 영화:")  # movie 값 출력
print(movie)


<class 'str'>
 ===> 추천된 영화:
드라마 장르라면 **「문라이트」(원제: *Moonlight*, 2016)**를 강력히 추천합니다.  
바리 젠킨스 감독의 이 작품은 성장, 정체성, 사랑을 섬세하게 그려낸 3부작 구성의 영화로, 미국 흑인 사회의 현실과 한 개인의 내면 성장을 시적이고도 절제된 화면으로 담아냅니다. 제88회 아카데미 시상식에서 작품상을 비롯해 3관왕에 오르며 역사를 썼죠.  
잔잔하면서도 울림이 큰 영화를 원하신다면, 「문라이트」를 보는 순간이야말로 ‘드라마’ 본연의 힘을 느낄 수 있는 경험이 될 겁니다.


In [ ]:
# 체인 2: 줄거리 요약 (입력: 영화 제목 → 출력: 줄거리)
chain2 = (
    {"movie": chain1}  # chain1의 출력을 movie 변수로 전달
    | prompt2
    | llm
    | StrOutputParser()
)
print(chain2)

# 실행: "SF" 장르의 영화 추천 및 줄거리 요약
response = chain2.invoke({"genre": "Drama"})
print("\n🔹 영화 줄거리 요약:\n", response) 

first={
  movie: ChatPromptTemplate(input_variables=['genre'], input_types={}, partial_variables={}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['genre'], input_types={}, partial_variables={}, template='{genre} 장르에서 추천할 만한 영화를 한 편 알려주세요.'), additional_kwargs={})])
         | ChatOpenAI(client=<openai.resources.chat.completions.completions.Completions object at 0x000001CA8C94EC90>, async_client=<openai.resources.chat.completions.completions.AsyncCompletions object at 0x000001CA8C94FDA0>, root_client=<openai.OpenAI object at 0x000001CA8C94E7E0>, root_async_client=<openai.AsyncOpenAI object at 0x000001CA8C94F5C0>, model_name='moonshotai/kimi-k2-instruct-0905', temperature=0.7, model_kwargs={}, openai_api_key=SecretStr('**********'), openai_api_base='https://api.groq.com/openai/v1')
         | StrOutputParser()
} middle=[ChatPromptTemplate(input_variables=['movie'], input_types={}, partial_variables={}, messages=[HumanMessagePromptTemplate(prompt=PromptTempl

##### chain1과 chain2에서 영화 제목이 일관되게 전달 되도록 변경 ( 잘 동작하는 코드)

In [ ]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser

from pprint import pprint

# Step 1: 사용자가 입력한 장르에 따라 영화 추천
prompt1 = ChatPromptTemplate.from_template("{genre} 장르에서 추천할 만한 영화를 한 편 알려주세요.")

# Step 2: 추천된 영화의 줄거리를 요약
prompt2 = ChatPromptTemplate.from_template("{movie} 추천한 영화의 제목을 먼저 알려주시고, 줄을 바꾸어서 영화의 정보(제목,감독,캐스팅,줄거리)를 알려 주세요.")

# OpenAI 모델 사용
llm = ChatOpenAI(
    api_key=OPENAI_API_KEY,
    base_url="https://api.groq.com/openai/v1",  # Groq API 엔드포인트
    #model="meta-llama/llama-4-scout-17b-16e-instruct",  # Spring AI와 동일한 모델
    model="moonshotai/kimi-k2-instruct-0905",
    temperature=0.7
)

# 체인 1: 영화 추천 (입력: 장르 → 출력: 영화 제목)
chain1 = prompt1 | llm | StrOutputParser()

# 체인 2: 줄거리 요약 (입력: 영화 제목 → 출력: 줄거리)
chain2 = (
    {"movie": chain1}  # chain1의 출력을 movie 변수로 전달
    | prompt2
    | llm
    | StrOutputParser()
)

# 실행: "Drama" 장르의 영화 추천 및 줄거리 요약
response = chain2.invoke({"genre": "Drama"})
print("\n🔹 영화 줄거리 요약:")
pprint(response)